## 1. Execute imports

In [1]:
!pip install sklearn-genetic

You should consider upgrading via the 'c:\users\walter ortiz\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
from genetic_selection import GeneticSelectionCV
from sklearn.metrics import classification_report

In [3]:
data_glass = pd.read_csv('glass.csv', delimiter=',')
X = pd.read_csv('glass_formula_a.csv', delimiter=',')
y = data_glass['Type']


In [4]:
X

RI     Na    Mg    Al     Si     K    Ca    Ba   Fe  cos(RI+Na)  \
0    1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.00  0.0   -0.854113   
1    1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.00  0.0   -0.955232   
2    1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.00  0.0   -0.788898   
3    1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.00  0.0   -0.556771   
4    1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.00  0.0   -0.605388   
..       ...    ...   ...   ...    ...   ...   ...   ...  ...         ...   
209  1.51623  14.14  0.00  2.88  72.61  0.08  9.18  1.06  0.0   -0.998662   
210  1.51685  14.92  0.00  1.99  73.06  0.00  8.40  1.59  0.0   -0.745916   
211  1.52065  14.36  0.00  2.02  73.42  0.00  8.44  1.64  0.0   -0.985127   
212  1.51651  14.38  0.00  1.94  73.61  0.00  8.48  1.57  0.0   -0.982278   
213  1.51711  14.23  0.00  2.08  73.36  0.00  8.62  1.67  0.0   -0.999234   

     ...  cos(Si+K)  cos(Si+Ca)  cos(Si+Ba)  cos(Si+Fe)  cos(K+Ca)  cos(K+Ba)  \
0    ...  -0.914457    0.407201   -0.888546   -0.888546  -0.816902   0.998201   
1    ...  -0.578939    0.434414   -0.890037   -0.890037  -0.440377   0.886995   
2    ...  -0.432648    0.612633   -0.742924   -0.742924  -0.310785   0.924909   
3    ...  -0.603136    0.658924   -0.938212   -0.938212  -0.805203   0.841901   
4    ...  -0.196147    0.862094   -0.679754   -0.679754  -0.693271   0.852525   
..   ...        ...         ...         ...         ...        ...        ...   
209  ...  -0.907556    0.994110   -0.156778   -0.938212  -0.986455   0.417595   
210  ...  -0.694286    0.975589    0.732899   -0.694286  -0.519289  -0.019202   
211  ...  -0.396249    0.984095    0.943346   -0.396249  -0.553048  -0.069148   
212  ...  -0.215718    0.917682    0.976284   -0.215718  -0.585923   0.000796   
213  ...  -0.450591    0.955752    0.932968   -0.450591  -0.693271  -0.099041   

     cos(K+Fe)  cos(Ca+Ba)  cos(Ca+Fe)  cos(Ba+Fe)  
0     0.998201   -0.780846   -0.780846    1.000000  
1     0.886995    0.023979    0.023979    1.000000  
2     0.924909    0.073914    0.073914    1.000000  
3     0.841901   -0.357900   -0.357900    1.000000  
4     0.852525   -0.214342   -0.214342    1.000000  
..         ...         ...         ...         ...  
209   0.996802   -0.685707   -0.970191    0.488872  
210   1.000000   -0.844470   -0.519289   -0.019202  
211   1.000000   -0.792913   -0.553048   -0.069148  
212   1.000000   -0.810833   -0.585923    0.000796  
213   1.000000   -0.648471   -0.693271   -0.099041  

[214 rows x 45 columns]

## KNN

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

### Information gain

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', KNeighborsClassifier())])

search_space = [{'selector__k': ['all']},
                {'classifier': [KNeighborsClassifier()]}]

In [8]:
%%time
knn_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
knn_info.fit(X_train, y_train)

Wall time: 3.6 s


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x0000014960680160>)),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [KNeighborsClassifier()]}])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)

In [13]:
selector = GeneticSelectionCV(knn_info,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [ ]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                	std                            	min             	max             
0  	50    	[  nan  4.62   nan]	[      nan  2.373942       nan]	[ nan   1.  nan]	[ nan   9.  nan]


In [ ]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

In [ ]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

In [ ]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

In [12]:
print(cross_val_score(knn_info, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.630952380952381


In [21]:
selector.n_features_

8

### Variance threshold

In [13]:
from sklearn.feature_selection import VarianceThreshold

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', KNeighborsClassifier())])

search_space = [{'classifier': [KNeighborsClassifier()]}]

In [15]:
%%time
knn_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
knn_variance.fit(X_train, y_train)

Wall time: 72.8 ms


GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid=[{'classifier': [KNeighborsClassifier()]}],
             scoring='accuracy')

In [25]:
selector = GeneticSelectionCV(knn_variance,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [26]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.570971  5.54      0.118934]	[ 0.075932  2.787185  0.025245]	[ 0.365238  1.        0.068454]	[ 0.655714  9.        0.186105]
1  	31    	[-799.421667    6.9       800.122427]	[ 2713.102535     2.443358  2712.895891]	[-10000.            2.            0.090282]	[     0.692381     13.        10000.      ]
2  	28    	[-1999.4884      7.36     2000.10265]	[ 4000.2558       2.330322  3999.948675]	[-10000.            2.            0.080255]	[     0.692381     13.        10000.      ]
3  	25    	[-1199.420152     7.54      1200.111608]	[ 3249.829485     2.031846  3249.574148]	[-10000.            2.            0.091598]	[     0.692381     13.        10000.      ]
4  	29    	[-1199.417867     7.58      1200.109561]	[ 3249.830329     1.877125  3249.574904]	[-10000.            4.            0.092313]	[    

In [27]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['Mg',
 'K',
 'cos(RI+Al)',
 'cos(Na+Si)',
 'cos(Mg+Ba)',
 'cos(Mg+Fe)',
 'cos(Al+K)',
 'cos(K+Ca)',
 'cos(K+Ba)']

In [28]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [29]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.69      0.89      0.77        27
           2       0.83      0.68      0.75        22
           3       0.00      0.00      0.00         7
           5       0.40      0.67      0.50         3
           6       0.00      0.00      0.00         2
           7       0.80      0.80      0.80        10

    accuracy                           0.69        71
   macro avg       0.45      0.51      0.47        71
weighted avg       0.65      0.69      0.66        71



In [16]:
print(cross_val_score(knn_variance, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.6506493506493507


### Chi-Square

In [18]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', KNeighborsClassifier())])

In [20]:
search_space = [{'classifier': [KNeighborsClassifier()]}]

In [21]:
%%time
knn_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
knn_chi.fit(X_train, y_train)

Wall time: 156 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x000001495F47F5E0>)),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid=[{'classifier': [KNeighborsClassifier()]}])

In [34]:
selector = GeneticSelectionCV(knn_chi,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [35]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.575295  4.92      0.112705]	[ 0.093556  2.322413  0.020364]	[ 0.280476  1.        0.070773]	[ 0.664762  9.        0.159096]
1  	28    	[-599.407676    6.68      600.10937 ]	[ 2375.018066     1.848675  2374.840786]	[-10000.            3.            0.043071]	[     0.720476     11.        10000.      ]
2  	24    	[-799.400743    7.        800.109361]	[ 2713.108705     1.897367  2712.899745]	[-10000.            3.            0.074803]	[     0.720476     12.        10000.      ]
3  	27    	[-799.392562    7.04      800.105555]	[ 2713.111117     1.87574   2712.900867]	[-10000.            2.            0.080318]	[     0.734762     11.        10000.      ]
4  	29    	[-799.37579     7.36      800.113056]	[ 2713.116063     1.571751  2712.898655]	[-10000.            4.            0.080318]	[     0.734

In [36]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['RI', 'Mg', 'Ba', 'cos(RI+Ba)', 'cos(Mg+Fe)', 'cos(Al+Fe)']

In [37]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [38]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.65      0.63      0.64        27
           2       0.53      0.73      0.62        22
           3       0.00      0.00      0.00         7
           5       0.50      0.67      0.57         3
           6       0.00      0.00      0.00         2
           7       0.89      0.80      0.84        10

    accuracy                           0.61        71
   macro avg       0.43      0.47      0.45        71
weighted avg       0.56      0.61      0.58        71



In [22]:
print(cross_val_score(knn_chi, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.6406926406926406


### Logistic Regression

In [23]:
from sklearn.linear_model import LogisticRegression

### Information gain

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', LogisticRegression())])

search_space = [{'selector__k': ['all']},
                {'classifier': [LogisticRegression()]}]

In [26]:
%%time
logistic_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
logistic_info.fit(X_train, y_train)

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

Wall time: 4.78 s


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x0000014960680160>)),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [LogisticRegression()]}])

In [44]:
selector = GeneticSelectionCV(logistic_info,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [45]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.559705  4.86      0.095482]	[ 0.071412  2.457723  0.023479]	[ 0.391429  1.        0.047323]	[ 0.664762  9.        0.148941]
1  	32    	[-1399.479924     6.46      1400.083745]	[ 3470.080152     2.616945  3469.836526]	[-10000.            1.            0.045826]	[     0.693333     13.        10000.      ]
2  	23    	[-799.412552    6.28      800.091967]   	[ 2713.105222     2.227465  2712.904874]	[-10000.            2.            0.051981]	[     0.699048     12.        10000.      ]
3  	29    	[-399.367571    7.04      400.098642]   	[ 1959.720888     1.574293  1959.571659]	[-10000.            3.            0.069687]	[     0.70619     12.       10000.     ]   
4  	25    	[-599.368181    7.54      600.097848]   	[ 2375.028044     1.499467  2374.843697]	[-10000.            4.            0.074719]

In [46]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['Na',
 'Al',
 'Fe',
 'cos(RI+Mg)',
 'cos(RI+Si)',
 'cos(RI+Fe)',
 'cos(Na+K)',
 'cos(K+Ca)',
 'cos(K+Ba)']

In [47]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [48]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.65      0.81      0.72        27
           2       0.74      0.64      0.68        22
           3       0.00      0.00      0.00         7
           5       1.00      0.67      0.80         3
           6       0.33      0.50      0.40         2
           7       0.82      0.90      0.86        10

    accuracy                           0.68        71
   macro avg       0.59      0.59      0.58        71
weighted avg       0.64      0.68      0.65        71



In [27]:
print(cross_val_score(logistic_info, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_i

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

0.6359307359307358


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Variance threshold

In [28]:
from sklearn.feature_selection import VarianceThreshold

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', LogisticRegression())])

search_space = [{'classifier': [LogisticRegression()]}]

In [30]:
%%time
logistic_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
logistic_variance.fit(X_train, y_train)

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

Wall time: 307 ms


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'classifier': [LogisticRegression()]}],
             scoring='accuracy')

In [52]:
selector = GeneticSelectionCV(logistic_variance,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [53]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.530114  4.86      0.092468]	[ 0.074592  2.584647  0.027193]	[ 0.357619  1.        0.041429]	[ 0.642857  9.        0.148174]
1  	28    	[-1399.494467     6.98      1400.093188]	[ 3470.074284     2.412385  3469.832716]	[-10000.            2.            0.047764]	[     0.642857     14.        10000.      ]
2  	30    	[-999.456467    7.48     1000.101049]   	[ 3000.181178     1.85731   2999.966317]	[-10000.            2.            0.062445]	[     0.670952     13.        10000.      ]
3  	30    	[-1399.46121      7.9       1400.097711]	[ 3470.087702     1.652271  3469.830891]	[-10000.            5.            0.067148]	[     0.670952     12.        10000.      ]
4  	26    	[-799.412943    7.68      800.102734]   	[ 2713.105107     1.462053  2712.901699]	[-10000.            5.            0.075587]

In [54]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['RI', 'cos(Mg+Ba)', 'cos(Al+Ba)', 'cos(Si+K)', 'cos(Si+Ca)', 'cos(K+Ca)']

In [55]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [56]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.66      0.85      0.74        27
           2       0.70      0.73      0.71        22
           3       0.00      0.00      0.00         7
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         2
           7       0.78      0.70      0.74        10

    accuracy                           0.65        71
   macro avg       0.36      0.38      0.36        71
weighted avg       0.57      0.65      0.61        71



C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
print(cross_val_score(logistic_variance, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_i

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

0.6547619047619047


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Chi-Square

In [32]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', LogisticRegression())])

In [34]:
search_space = [{'classifier': [LogisticRegression()]}]

In [35]:
%%time
logistic_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
logistic_chi.fit(X_train, y_train)

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

Wall time: 315 ms


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x000001495F47F5E0>)),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'classifier': [LogisticRegression()]}])

In [62]:
selector = GeneticSelectionCV(logistic_chi,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [63]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.494219  4.68      0.079494]	[ 0.072792  2.626328  0.022016]	[ 0.34381   1.        0.049175]	[ 0.62381   9.        0.147093]
1  	25    	[-399.467362    6.96      400.086224]	[ 1959.700519     2.116223  1959.574194]	[-10000.            1.            0.054338]	[     0.62381     11.       10000.     ]
2  	29    	[-2599.575124     8.14      2600.063757]	[ 4386.594285     1.969873  4386.304648]	[-10000.            3.            0.054638]	[     0.62381     12.       10000.     ]
3  	32    	[-999.463838    8.02     1000.081475]   	[ 3000.178721     1.348925  2999.972842]	[-10000.            5.            0.055451]	[     0.637143     12.        10000.      ]
4  	23    	[-1399.481819     8.26      1400.077806]	[ 3470.079387     1.947409  3469.838922]	[-10000.            3.            0.057398]	[     0.

In [64]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['Al',
 'cos(RI+Mg)',
 'cos(RI+Al)',
 'cos(Na+Si)',
 'cos(Mg+Ca)',
 'cos(Si+Ca)',
 'cos(K+Ca)',
 'cos(Ca+Ba)']

In [65]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [66]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.65      0.89      0.75        27
           2       0.71      0.68      0.70        22
           3       0.00      0.00      0.00         7
           5       0.25      0.33      0.29         3
           6       0.00      0.00      0.00         2
           7       0.78      0.70      0.74        10

    accuracy                           0.66        71
   macro avg       0.40      0.43      0.41        71
weighted avg       0.59      0.66      0.62        71



C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
print(cross_val_score(logistic_chi, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_i

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

0.6461038961038962


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

## SVM

In [37]:
from sklearn.svm import SVC

### Information gain

In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', SVC(kernel='linear'))])

search_space = [{'selector__k': ['all']},
                {'classifier': [SVC(kernel='linear')]}]

In [40]:
%%time
svm_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
svm_info.fit(X_train, y_train)

Wall time: 3.68 s


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x0000014960680160>)),
                                       ('classifier', SVC(kernel='linear'))]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [SVC(kernel='linear')]}])

In [98]:
selector = GeneticSelectionCV(svm_info,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [99]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.571095  5.78      0.090179]	[ 0.079173  2.729762  0.021151]	[ 0.378571  1.        0.040079]	[ 0.679524  9.        0.125809]
1  	32    	[-999.430171    7.74     1000.088438]	[ 3000.189943     2.076632  2999.970521]	[-10000.            4.            0.059455]	[     0.693333     16.        10000.      ]
2  	23    	[-1999.47379      8.24      2000.072428]	[ 4000.263105     1.668053  3999.963786]	[-10000.            5.            0.059455]	[     0.693333     12.        10000.      ]
3  	33    	[-799.385848    7.96      800.080455]   	[ 2713.113097     1.130664  2712.908268]	[-10000.            5.            0.059341]	[     0.70619     10.       10000.     ]   
4  	29    	[-1599.432743     8.36      1600.068555]	[ 3666.308127     1.609472  3666.030636]	[-10000.            5.            0.057978]	[ 

In [100]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['Mg',
 'Al',
 'Ca',
 'cos(Na+Al)',
 'cos(Na+Ba)',
 'cos(Al+Ba)',
 'cos(K+Ca)',
 'cos(K+Fe)']

In [101]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [102]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.65      0.81      0.72        27
           2       0.73      0.73      0.73        22
           3       0.00      0.00      0.00         7
           5       0.40      0.67      0.50         3
           6       0.00      0.00      0.00         2
           7       0.78      0.70      0.74        10

    accuracy                           0.66        71
   macro avg       0.43      0.48      0.45        71
weighted avg       0.60      0.66      0.62        71



C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
print(cross_val_score(svm_info, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.6318181818181817


### Variance threshold

In [42]:
from sklearn.feature_selection import VarianceThreshold

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', SVC(kernel='linear'))])

search_space = [{'classifier': [SVC(kernel='linear')]}]

In [44]:
%%time
svm_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
svm_variance.fit(X_train, y_train)

Wall time: 65.7 ms


GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', SVC(kernel='linear'))]),
             param_grid=[{'classifier': [SVC(kernel='linear')]}],
             scoring='accuracy')

In [106]:
selector = GeneticSelectionCV(svm_variance,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [107]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.524695  4.68      0.086804]	[ 0.071955  2.556873  0.024905]	[ 0.378571  1.        0.041853]	[ 0.650952  9.        0.145787]
1  	32    	[ 0.58141   6.14      0.098788]	[ 0.062165  2.0001    0.021789]	[ 0.45381   2.        0.047764]	[ 0.684762  9.        0.139305]
2  	23    	[-1199.4544       6.88      1200.091306]	[ 3249.816838     2.285957  3249.581645]	[-10000.            2.            0.041918]	[     0.691905     12.        10000.      ]
3  	29    	[-999.422181    6.86     1000.092053]   	[ 3000.192607     1.990075  2999.969316]	[-10000.            4.            0.065424]	[     0.691905     11.        10000.      ]
4  	35    	[-199.37059     6.52      200.097065]   	[ 1400.089916     1.824719  1399.986134]	[-10000.            3.            0.056215]	[     0.699048     10.        10000.     

In [108]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['Mg',
 'Al',
 'cos(Na+Si)',
 'cos(Mg+Fe)',
 'cos(Al+Ca)',
 'cos(Si+Fe)',
 'cos(K+Ca)',
 'cos(K+Ba)',
 'cos(Ca+Ba)']

In [109]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [110]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.66      0.85      0.74        27
           2       0.68      0.59      0.63        22
           3       0.00      0.00      0.00         7
           5       0.40      0.67      0.50         3
           6       0.00      0.00      0.00         2
           7       0.73      0.80      0.76        10

    accuracy                           0.65        71
   macro avg       0.41      0.48      0.44        71
weighted avg       0.58      0.65      0.61        71



C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
print(cross_val_score(svm_variance, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.6658008658008658


### Chi-Square

In [46]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', SVC())])

In [48]:
search_space = [{'classifier': [SVC()]}]

In [49]:
%%time
svm_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
svm_chi.fit(X_train, y_train)

Wall time: 80.8 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x000001495F47F5E0>)),
                                       ('classifier', SVC())]),
             param_grid=[{'classifier': [SVC()]}])

In [115]:
selector = GeneticSelectionCV(svm_chi,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [116]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.562038  4.68      0.09123 ]	[ 0.061244  2.469332  0.02282 ]	[ 0.357143  1.        0.040473]	[ 0.664286  9.        0.135651]
1  	28    	[-399.411638    5.7       400.099088]	[ 1959.711893     2.376973  1959.571568]	[-10000.            1.            0.069556]	[     0.670952     11.        10000.      ]
2  	28    	[ 0.634695  6.72      0.106998]      	[ 0.04046   1.510497  0.02296 ]         	[ 0.448095  3.        0.058633]            	[ 0.691429  9.        0.148632]            
3  	36    	[-599.384333    7.68      600.100391]	[ 2375.023963     1.702234  2374.843054]	[-10000.            4.            0.076659]	[     0.691429     15.        10000.      ]
4  	29    	[-1199.41399      7.86      1200.091223]	[ 3249.83176      1.918437  3249.581676]	[-10000.            3.            0.072282]	[     0.

In [117]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['Na',
 'Al',
 'cos(RI+Na)',
 'cos(Na+Al)',
 'cos(Na+Ca)',
 'cos(Mg+Al)',
 'cos(Al+K)',
 'cos(K+Ca)']

In [118]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [119]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.65      0.89      0.75        27
           2       0.68      0.59      0.63        22
           3       0.00      0.00      0.00         7
           5       0.50      0.67      0.57         3
           6       0.00      0.00      0.00         2
           7       0.80      0.80      0.80        10

    accuracy                           0.66        71
   macro avg       0.44      0.49      0.46        71
weighted avg       0.59      0.66      0.62        71



C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [50]:
print(cross_val_score(svm_chi, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.6467532467532469


## Naive Bayes

In [53]:
from sklearn.naive_bayes import GaussianNB

### Information gain

In [54]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', GaussianNB())])

search_space = [{'selector__k': ['all']},
                {'classifier': [GaussianNB()]}]

In [56]:
%%time
nb_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_info.fit(X_train, y_train)

Wall time: 3.86 s


GridSearchCV(estimator=Pipeline(steps=[('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x0000014960680160>)),
                                       ('classifier', GaussianNB())]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [GaussianNB()]}])

In [71]:
selector = GeneticSelectionCV(nb_info,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [72]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.434257  5.1       0.099088]	[ 0.11243   2.677686  0.031422]	[ 0.174762  1.        0.053461]	[ 0.567143  9.        0.188399]
1  	31    	[-599.52861     5.88      600.108896]	[ 2374.987513     2.84      2374.840906]	[-10000.            1.            0.061657]	[     0.623333     12.        10000.      ]
2  	26    	[-999.514267    7.1      1000.10856 ]	[ 3000.161911     2.137756  2999.963813]	[-10000.            1.            0.069177]	[     0.644762     13.        10000.      ]
3  	29    	[-1199.511524     7.36      1200.10298 ]	[ 3249.795744     2.161111  3249.577334]	[-10000.            2.            0.061686]	[     0.644762     14.        10000.      ]
4  	33    	[-1599.504276     7.52      1600.097884]	[ 3666.276908     1.992386  3666.017836]	[-10000.            4.            0.082869]	[    

In [73]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['Al', 'cos(Mg+K)', 'cos(Al+Si)', 'cos(Al+Fe)', 'cos(K+Ba)', 'cos(Ca+Fe)']

In [74]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [75]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.71      0.89      0.79        27
           2       0.76      0.86      0.81        22
           3       0.00      0.00      0.00         7
           5       0.67      0.67      0.67         3
           6       0.00      0.00      0.00         2
           7       0.89      0.80      0.84        10

    accuracy                           0.75        71
   macro avg       0.50      0.54      0.52        71
weighted avg       0.66      0.75      0.70        71



C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [57]:
print(cross_val_score(nb_info, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.44307359307359306


### Variance threshold

In [58]:
from sklearn.feature_selection import VarianceThreshold

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', GaussianNB())])

search_space = [{'classifier': [GaussianNB()]}]

In [60]:
nb_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
nb_variance.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', GaussianNB())]),
             param_grid=[{'classifier': [GaussianNB()]}], scoring='accuracy')

In [80]:
selector = GeneticSelectionCV(nb_variance,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [81]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.451867  4.88      0.099629]	[ 0.096709  2.650585  0.030759]	[ 0.217619  1.        0.036642]	[ 0.609048  9.        0.170958]
1  	33    	[-399.503476    5.96      400.108545]	[ 1959.693147     2.568735  1959.569638]	[-10000.           1.           0.05472]	[     0.617143     12.        10000.      ]
2  	38    	[-599.485752    6.9       600.108186]	[ 2374.99834      1.920937  2374.841085]	[-10000.            3.            0.057666]	[     0.657619     13.        10000.      ]
3  	27    	[-999.476971    7.64     1000.106611]	[ 3000.174343     1.786169  2999.964463]	[-10000.            4.            0.067848]	[     0.657619     12.        10000.      ]
4  	30    	[-1199.474619     7.94      1200.105245]	[ 3249.809372     1.58      3249.576498]	[-10000.           4.           0.06708]   	[     0.679

In [82]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['Na', 'cos(RI+Ca)', 'cos(Mg+K)', 'cos(Al+Fe)', 'cos(K+Ba)', 'cos(Ca+Fe)']

In [83]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [84]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.62      0.85      0.72        27
           2       0.72      0.59      0.65        22
           3       0.00      0.00      0.00         7
           5       0.33      0.67      0.44         3
           6       0.00      0.00      0.00         2
           7       0.78      0.70      0.74        10

    accuracy                           0.63        71
   macro avg       0.41      0.47      0.43        71
weighted avg       0.58      0.63      0.60        71



C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [61]:
print(cross_val_score(nb_variance, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.44307359307359306


### Chi-Square

In [62]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', GaussianNB())])

In [64]:
search_space = [{'classifier': [GaussianNB()]}]

In [65]:
%%time
nb_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_chi.fit(X_train, y_train)

Wall time: 67.5 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x000001495F47F5E0>)),
                                       ('classifier', GaussianNB())]),
             param_grid=[{'classifier': [GaussianNB()]}])

In [89]:
selector = GeneticSelectionCV(nb_chi,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [90]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.449667  5.14      0.103077]	[ 0.099301  2.366516  0.026369]	[ 0.222857  1.        0.048841]	[ 0.629524  9.        0.152961]
1  	26    	[-399.489552    6.        400.106688]	[ 1959.69599      2.391652  1959.570017]	[-10000.            1.            0.059811]	[     0.663333     10.        10000.      ]
2  	35    	[-1399.522267     6.9       1400.100728]	[ 3470.063068     2.247221  3469.829674]	[-10000.            2.            0.062751]	[     0.664286     11.        10000.      ]
3  	24    	[-599.444457    6.82      600.123229]   	[ 2375.008774     2.046363  2374.837284]	[-10000.            3.            0.073742]	[     0.670952     11.        10000.      ]
4  	21    	[-399.401352    6.16      400.128505]   	[ 1959.713994     2.265921  1959.565563]	[-10000.            3.            0.083048]	[ 

In [91]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['Na',
 'Al',
 'cos(RI+Ca)',
 'cos(Mg+Al)',
 'cos(Mg+K)',
 'cos(Al+Ba)',
 'cos(K+Ba)',
 'cos(Ca+Fe)']

In [92]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [93]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.71      0.89      0.79        27
           2       0.78      0.82      0.80        22
           3       1.00      0.14      0.25         7
           5       0.67      0.67      0.67         3
           6       0.00      0.00      0.00         2
           7       0.89      0.80      0.84        10

    accuracy                           0.75        71
   macro avg       0.67      0.55      0.56        71
weighted avg       0.76      0.75      0.72        71



In [66]:
print(cross_val_score(nb_chi, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.44307359307359306
